### Random Forest Regressor on AQI

In [1]:
import pandas as pd
import numpy as np

In [4]:
random_data =pd.read_csv("C:/Users/Viswanath/shalini_DSPractice/project1_AQI/Data/Real-Data/Real_Combine.csv")

In [5]:
random_data.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


###  Here all columns are continuous ,no need of conversion ,next step is check Null values

In [6]:
random_data.isnull().values.any()

True

In [7]:
random_data.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    1
dtype: int64

#### Null value at target column PM 2.5 So remove the row

In [8]:
random_data=random_data.dropna()

In [9]:
random_data.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    0
dtype: int64

In [10]:
X= random_data.iloc[:,:-1]

In [11]:
Y= random_data.iloc[:,-1:]

#### All other preprocessing steps similar to prior modelling.Here I directly go to Ramdom forest model part

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV,train_test_split


In [13]:
RF_regressor =RandomForestRegressor() ### With all default parameters

In [14]:
Train_x,Test_x,Train_y,Test_y = train_test_split(X,Y,test_size=0.2,random_state=0)

In [15]:
RF_regressor.fit(Train_x,Train_y)

C:\Users\Viswanath\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Viswanath\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [16]:
print("R_Square on Train data",RF_regressor.score(Train_x,Train_y))

R_Square on Train data 0.940322160439672


In [17]:
print("R_Square on Test data",RF_regressor.score(Test_x,Test_y))

R_Square on Test data 0.4191798334557557


In [18]:
from sklearn.model_selection import cross_val_score
score =cross_val_score(RF_regressor,X,Y,cv=6)

C:\Users\Viswanath\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Viswanath\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Viswanath\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Viswanath\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A co

In [19]:
score.mean()

0.47638407762043466

In [20]:
prediction_RF = RF_regressor.predict(Test_x)

### Start with Hyper parameter tunning ON RF using RamdomizedSearchCV

In [21]:
### Sample code space
n_estimatrors=[int(x) for x in np.linspace(start=100,stop=2000,num=15)]
n_estimatrors

[100,
 235,
 371,
 507,
 642,
 778,
 914,
 1050,
 1185,
 1321,
 1457,
 1592,
 1728,
 1864,
 2000]

In [22]:
## Number of trees in random forest
n_estimators =[int(x) for x in np.linspace(start=100,stop=2000,num=15)]
# Number of features to consider at every split
n_maxfeatures =['auto','sqrt']
# Maximum number of levels in tree
n_maxdepth =[int(x) for x in np.linspace(start=5,stop=30,num=7)]
# Minimum number of samples required to split a node
n_minsample =[int(x) for x in np.linspace(start=5,stop=30,num=7)]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [23]:
paramters ={'n_estimators':n_estimators,'max_features':n_maxfeatures,'max_depth':n_maxdepth,
            'min_samples_split':n_minsample,'min_samples_leaf': min_samples_leaf}

In [24]:
RFhyper_regressor = RandomizedSearchCV(RF_regressor,param_distributions =paramters,cv=6,scoring='neg_mean_squared_error',
                                       verbose=2,n_jobs=-1,random_state=0,n_iter=100)

In [25]:
RFhyper_regressor.fit(Train_x,Train_y)

Fitting 6 folds for each of 100 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  5.8min finished
C:\Users\Viswanath\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Viswanath\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=6, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=10, n_jobs=None,
                                                   oob_score=False,
                                                   random_state=N...


In [26]:
RFhyper_regressor.best_params_

{'n_estimators': 507,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 9}

In [27]:
RFhyper_regressor.best_score_

-2387.3848597021665

In [28]:
prediction_RFhyper=RFhyper_regressor.predict(Test_x)

In [29]:
from sklearn import metrics
print('MAE RFHyper :', metrics.mean_absolute_error(Test_y, prediction_RFhyper))
print('MSE RFHyper:', metrics.mean_squared_error(Test_y, prediction_RFhyper))
print('RMSE RFHyper:', np.sqrt(metrics.mean_squared_error(Test_y, prediction_RFhyper)))

MAE RFHyper : 35.73325010232421
MSE RFHyper: 2597.1392167223607
RMSE RFHyper: 50.962135127193804


In [30]:
from sklearn import metrics
### Random forest without hyper parameters
print('MAE RF :', metrics.mean_absolute_error(Test_y, prediction_RF))
print('MSE RF:', metrics.mean_squared_error(Test_y, prediction_RF))
print('RMSE RF:', np.sqrt(metrics.mean_squared_error(Test_y, prediction_RF)))

MAE RF : 41.689511986301376
MSE RF: 3495.0357407355878
RMSE RF: 59.1188272949962


In [31]:
### Store the model
import pickle
file =open('RandomforestAQI.pkl','wb')
pickle.dump(RFhyper_regressor,file)

In [34]:
import matplotlib
print(matplotlib.__version__)

3.1.1
